# Análisis de Autores Activos: Mapping Review IA y ML en Educación Matemática K-12

**MQ2: ¿Quiénes son los autores más activos en este campo?**

Este notebook analiza los autores más productivos y sus patrones de colaboración en el campo de IA y ML en educación matemática K-12.

## 1. Configuración del Entorno

In [ ]:
# Instalación de dependencias
!pip install pandas numpy matplotlib seaborn plotly networkx

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## 2. Carga de Datos desde GitHub

In [ ]:
# Cargar el dataset desde GitHub
# IMPORTANTE: Cambiar la URL por tu repositorio real
url = "https://raw.githubusercontent.com/mlproyecto/doctorado/main/MappingReview.csv"
df = pd.read_csv(url, sep=';', encoding='latin-1')

print(f"Dataset cargado: {df.shape[0]} filas y {df.shape[1]} columnas")
print("\nPrimeras 5 filas:")
df.head()

## 3. Análisis de Autores Activos (MQ2)

In [ ]:
# Función para extraer autores individuales
def extract_authors(authors_str):
    if pd.isna(authors_str):
        return []
    
    # Limpiar el string de autores
    authors_str = str(authors_str).strip()
    
    # Dividir por punto y coma y limpiar
    authors = [author.strip() for author in authors_str.split(';')]
    
    # Remover entradas vacías
    authors = [author for author in authors if author]
    
    return authors

# Aplicar extracción de autores
df['Authors_List'] = df['Author(s)'].apply(extract_authors)

print("Extracción de autores completada")
print(f"Publicaciones con autores extraídos: {len(df[df['Authors_List'].apply(len) > 0])}")

In [ ]:
# Contar frecuencia de autores
all_authors = []
for authors_list in df['Authors_List']:
    all_authors.extend(authors_list)

author_counts = Counter(all_authors)

print("=== AUTORES MÁS FRECUENTES ===")
for author, count in author_counts.most_common(15):
    print(f"{author}: {count} publicaciones")

print(f"\nTotal de autores únicos: {len(author_counts)}")
print(f"Total de apariciones de autores: {sum(author_counts.values())}")

In [ ]:
# Gráfico de barras para autores más frecuentes
top_authors = dict(author_counts.most_common(10))

plt.figure(figsize=(15, 8))
bars = plt.bar(range(len(top_authors)), top_authors.values(), color='lightblue', alpha=0.7)
plt.xlabel('Autor', fontsize=14)
plt.ylabel('Número de Publicaciones', fontsize=14)
plt.title('Autores Más Productivos', fontsize=16, fontweight='bold')
plt.xticks(range(len(top_authors)), list(top_authors.keys()), rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')

# Agregar valores en las barras
for i, (bar, count) in enumerate(zip(bars, top_authors.values())):
    plt.text(i, count + 0.1, str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico interactivo con Plotly
fig = px.bar(x=list(top_authors.keys()), y=list(top_authors.values()),
              title='Autores Más Productivos',
              labels={'x': 'Autor', 'y': 'Número de Publicaciones'})

fig.update_layout(
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

fig.show()

## 4. Análisis de Colaboraciones

In [ ]:
# Crear red de colaboraciones
G = nx.Graph()

# Agregar nodos (autores)
for author in author_counts.keys():
    G.add_node(author, weight=author_counts[author])

# Agregar edges (colaboraciones)
for authors_list in df['Authors_List']:
    if len(authors_list) > 1:
        for i in range(len(authors_list)):
            for j in range(i + 1, len(authors_list)):
                author1 = authors_list[i]
                author2 = authors_list[j]
                
                if G.has_edge(author1, author2):
                    G[author1][author2]['weight'] += 1
                else:
                    G.add_edge(author1, author2, weight=1)

print(f"Red de colaboraciones creada")
print(f"Número de nodos (autores): {G.number_of_nodes()}")
print(f"Número de edges (colaboraciones): {G.number_of_edges()}")

In [ ]:
# Análisis de centralidad
centrality_degree = nx.degree_centrality(G)
centrality_betweenness = nx.betweenness_centrality(G)
centrality_closeness = nx.closeness_centrality(G)

# Top autores por centralidad
top_degree = sorted(centrality_degree.items(), key=lambda x: x[1], reverse=True)[:10]
top_betweenness = sorted(centrality_betweenness.items(), key=lambda x: x[1], reverse=True)[:10]
top_closeness = sorted(centrality_closeness.items(), key=lambda x: x[1], reverse=True)[:10]

print("=== TOP 10 AUTORES POR CENTRALIDAD ===")
print("\nPor Grado:")
for author, centrality in top_degree:
    print(f"{author}: {centrality:.3f}")

print("\nPor Intermediación:")
for author, centrality in top_betweenness:
    print(f"{author}: {centrality:.3f}")

print("\nPor Cercanía:")
for author, centrality in top_closeness:
    print(f"{author}: {centrality:.3f}")

In [ ]:
# Gráfico de la red de colaboraciones (versión simplificada)
# Usar solo los autores más conectados para visualización
top_connected = sorted(G.degree(), key=lambda x: x[1], reverse=True)[:20]
subgraph = G.subgraph([author for author, degree in top_connected])

plt.figure(figsize=(15, 10))
pos = nx.spring_layout(subgraph, k=3, iterations=50)

# Dibujar nodos
node_sizes = [author_counts[author] * 100 for author in subgraph.nodes()]
nx.draw_networkx_nodes(subgraph, pos, node_size=node_sizes, node_color='lightblue', alpha=0.7)

# Dibujar edges
edge_weights = [subgraph[u][v]['weight'] for u, v in subgraph.edges()]
nx.draw_networkx_edges(subgraph, pos, width=edge_weights, alpha=0.5, edge_color='gray')

# Dibujar etiquetas
nx.draw_networkx_labels(subgraph, pos, font_size=8, font_weight='bold')

plt.title('Red de Colaboraciones entre Autores', fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## 5. Análisis de Tamaños de Equipo

In [ ]:
# Análisis de tamaños de equipo
team_sizes = df['Authors_List'].apply(len)
team_size_counts = team_sizes.value_counts().sort_index()

print("=== DISTRIBUCIÓN DE TAMAÑOS DE EQUIPO ===")
for size, count in team_size_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{size} autor(es): {count} publicaciones ({percentage:.1f}%)")

print(f"\nPromedio de autores por publicación: {team_sizes.mean():.2f}")
print(f"Mediana de autores por publicación: {team_sizes.median():.0f}")
print(f"Máximo de autores por publicación: {team_sizes.max()}")
print(f"Mínimo de autores por publicación: {team_sizes.min()}")

In [ ]:
# Gráfico de distribución de tamaños de equipo
plt.figure(figsize=(12, 8))
bars = plt.bar(team_size_counts.index, team_size_counts.values, color='lightgreen', alpha=0.7)
plt.xlabel('Número de Autores', fontsize=14)
plt.ylabel('Número de Publicaciones', fontsize=14)
plt.title('Distribución de Tamaños de Equipo', fontsize=16, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Agregar valores en las barras
for i, (bar, count) in enumerate(zip(bars, team_size_counts.values)):
    plt.text(team_size_counts.index[i], count + 0.5, str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Resumen y Conclusiones

In [ ]:
# Generar resumen ejecutivo
print("=== RESUMEN EJECUTIVO ===\n")

print(f"📊 Total de autores únicos: {len(author_counts)}")
print(f"📝 Autor más productivo: {author_counts.most_common(1)[0][0]} ({author_counts.most_common(1)[0][1]} publicaciones)")
print(f"👥 Promedio de autores por publicación: {team_sizes.mean():.2f}")
print(f"🔗 Colaboraciones únicas: {G.number_of_edges()}")

# Análisis de colaboración
solo_authors = len(df[df['Authors_List'].apply(len) == 1])
collaborative = len(df[df['Authors_List'].apply(len) > 1])
collaboration_percentage = (collaborative / len(df)) * 100

print(f"📈 Publicaciones colaborativas: {collaborative} ({collaboration_percentage:.1f}%)")
print(f"📉 Publicaciones individuales: {solo_authors} ({100 - collaboration_percentage:.1f}%)")

# Autor más central
most_central = top_degree[0]
print(f"🏆 Autor más central: {most_central[0]} (centralidad: {most_central[1]:.3f})")

print("\n=== CONCLUSIONES ===")
print("1. Hay autores claramente más productivos en el campo")
print("2. Existe una red de colaboraciones significativa")
print("3. La mayoría de publicaciones son colaborativas")
print("4. Hay autores que actúan como conectores en la red")